In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
widths = [1]*32
df = pd.read_fwf('digitdata/optdigits-orig_train.txt', widths=widths, header=None, skiprows=lambda x : (x+1) % 33 == 0)
# df
df_Y = pd.read_fwf('digitdata/optdigits-orig_train.txt', widths=[2], header=None, skiprows=lambda x : (x+1) % 33 != 0)
print(len(df_Y))

2436


In [3]:
df_trainX = pd.DataFrame(df.values.reshape(2436, -1))
df_trainY = df_Y
df_trainY.columns = ['label']
# df_Y

In [4]:
n = 2
m = 2
length = int(32*32/n/m)

df_disjoint = pd.DataFrame(np.nan, index=range(2436), columns=range(length))

In [7]:
for idx, image in df_trainX.iterrows():
    for i in range(length):
        temp = ''
        c = 0
        if (i%(32/m) == 0 and not i == 0):
            c += 32
        for k in range(n):
            for l in range(m):
                temp += str(image[m*i+l+c+ 32*k])
        df_disjoint.iloc[idx, i] = int(temp, 2)


In [8]:
df = pd.read_fwf('digitdata/optdigits-orig_test.txt', widths=widths, header=None, skiprows=lambda x : (x+1) % 33 == 0)
df_Y = pd.read_fwf('digitdata/optdigits-orig_test.txt', widths=[2], header=None, skiprows=lambda x : (x+1) % 33 != 0)


In [9]:
df_testX = pd.DataFrame(df.values.reshape(444, -1))
df_testY = df_Y
df_testY.columns = ['label']

In [10]:
df_test = pd.DataFrame(np.nan, index=range(444), columns=range(length))

for idx, image in df_testX.iterrows():
    for i in range(length):
        temp = ''
        c = 0
        if (i%(32/m) == 0 and not i == 0):
            c += 32
        for k in range(n):
            for l in range(m):
                temp += str(image[m*i+l+c+ 32*k])
        df_test.iloc[idx, i] = int(temp, 2)


In [11]:
class_count = np.array((df_trainY['label'].value_counts().sort_index()))


In [12]:
class_log_prior = np.log(class_count) - np.log(len(df_trainY))


In [13]:
#feature_count.append(np.array([[[sum(df_disjoint[df_trainY['label'] == i].iloc[:,j] ) for i in range(10)] for j in range(256)]).T)


feature_MF = []
for val in range(2**(n*m)):
    feature_makeval = np.copy(df_disjoint)
    feature_makeval[df_disjoint != val] = 0
    feature_makeval[df_disjoint == val] = 1
#     only count the frenqucy of certain feature whose value == val
    feature_count = np.array([[sum(feature_makeval[df_trainY['label'] == i][:,j]) for i in range(10)] for j in range(length)]).T
    feature_MF.append(feature_count)
feature_MF = np.swapaxes(feature_MF, 0, 1)
feature_MF = np.swapaxes(feature_MF, 1, 2)
feature_MF = np.reshape(feature_MF, (10, (2**(n*m))*length))


In [14]:
def calc_feature_log_prob(feature_MF, k=1):
    smoothed_fc = feature_MF + k
    smoothed_cc = smoothed_fc.sum(axis=1)
    return (np.log(smoothed_fc) - np.log(smoothed_cc.reshape(-1, 1)))

In [15]:
feature_log_prob = [calc_feature_log_prob(feature_MF, k=0.1), \
                    calc_feature_log_prob(feature_MF, k=1), \
                    calc_feature_log_prob(feature_MF, k=5), \
                    calc_feature_log_prob(feature_MF, k=10)]

In [16]:
test_MF = np.zeros((444 , length, (2**(n*m))  ))
for i in range(444):
    for j in range(length):
        for k in range(2**(n*m)):
            test_MF[i, j, k] = 1 if df_test.iloc[i, j] == k else 0

In [17]:
test_MF = np.reshape(test_MF, (444, length * (2**(n*m))))        

In [18]:
joint_log_likelihood = [test_MF @ feature_log_prob[0].T + class_log_prior, \
                        test_MF @ feature_log_prob[1].T + class_log_prior, \
                        test_MF @ feature_log_prob[2].T + class_log_prior, \
                        test_MF @ feature_log_prob[3].T + class_log_prior]
# joint_log_likelihood

In [19]:
pred = [np.argmax(joint_log_likelihood[0], axis=1), \
        np.argmax(joint_log_likelihood[1], axis=1), \
        np.argmax(joint_log_likelihood[2], axis=1), \
        np.argmax(joint_log_likelihood[3], axis=1)]

In [20]:
smooth_k = [0.1, 1, 5, 10]
for i in range(4):
    y_pred = pd.Series(pred[i])
    df_confusion = pd.crosstab(df_testY['label'], y_pred)

    conf_percent = df_confusion.values / df_confusion.sum(axis=1).values.reshape(-1,1)
    df_conf_percent = round(pd.DataFrame(conf_percent).rename_axis('Actual').rename_axis('Prediction', axis=1), 4)*100
    print('Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = {})'.format(smooth_k[i]))
    display(df_conf_percent)
    display(pd.DataFrame(np.diag(df_conf_percent), columns=['Accuracy']))
    print('Average Accuracy: {}'.format(np.mean(np.diag(df_conf_percent))))
    print('\n')
# df_conf_percent

Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = 0.1)


Prediction,0,1,2,3,4,5,6,7,8,9
Actual,,,,,,,,,,
0,97.22,2.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,97.78,0.00,0.00,0.00,0.00,0.00,2.22,0.00,0.00
2,0.00,0.00,70.73,14.63,0.00,0.00,0.00,4.88,0.00,9.76
3,0.00,0.00,6.06,78.79,0.00,0.00,0.00,9.09,6.06,0.00
4,1.69,6.78,0.00,0.00,77.97,0.00,1.69,0.00,3.39,8.47
5,0.00,0.00,0.00,0.00,0.00,96.55,1.72,0.00,0.00,1.72
6,0.00,0.00,2.33,0.00,0.00,0.00,97.67,0.00,0.00,0.00
7,0.00,4.26,0.00,12.77,0.00,2.13,0.00,80.85,0.00,0.00
8,0.00,0.00,2.50,2.50,0.00,0.00,0.00,2.50,87.50,5.00


,Accuracy
0,97.22
1,97.78
2,70.73
3,78.79
4,77.97
5,96.55
6,97.67
7,80.85
8,87.50
9,83.33


Average Accuracy: 86.839


Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = 1)


Prediction,0,1,2,3,4,5,6,7,8,9
Actual,,,,,,,,,,
0,97.22,2.78,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0.00,97.78,0.00,0.00,0.00,0.00,0.00,2.22,0.00,0.00
2,0.00,0.00,70.73,14.63,0.00,0.00,0.00,4.88,0.00,9.76
3,0.00,0.00,6.06,78.79,0.00,0.00,0.00,9.09,6.06,0.00
4,1.69,8.47,0.00,0.00,74.58,0.00,1.69,0.00,3.39,10.17
5,0.00,0.00,0.00,0.00,0.00,96.55,1.72,0.00,0.00,1.72
6,0.00,0.00,2.33,0.00,0.00,0.00,97.67,0.00,0.00,0.00
7,0.00,4.26,0.00,17.02,0.00,0.00,0.00,78.72,0.00,0.00
8,0.00,2.50,2.50,2.50,0.00,2.50,0.00,2.50,82.50,5.00


...

,Accuracy
0,97.22
1,97.78
2,70.73
3,78.79
4,67.80
5,94.83
6,97.67
7,72.34
8,75.00
9,76.19


Average Accuracy: 82.835


Test set Confusion Matrix: Actual vs. Prediction (Smoothing value k = 10)


Prediction,0,1,2,3,4,5,6,7,8,9
Actual,,,,,,,,,,
0,97.22,2.78,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00
1,0.00,97.78,0.00,0.00,0.0,0.00,0.00,2.22,0.00,0.00
2,2.44,0.00,70.73,14.63,0.0,0.00,0.00,4.88,0.00,7.32
3,0.00,3.03,6.06,78.79,0.0,0.00,0.00,9.09,3.03,0.00
4,1.69,11.86,0.00,0.00,66.1,0.00,8.47,0.00,3.39,8.47
5,0.00,3.45,0.00,0.00,0.0,93.10,3.45,0.00,0.00,0.00
6,0.00,0.00,2.33,0.00,0.0,0.00,97.67,0.00,0.00,0.00
7,0.00,6.38,0.00,27.66,0.0,0.00,0.00,65.96,0.00,0.00
8,0.00,5.00,5.00,7.50,0.0,0.00,0.00,2.50,67.50,12.50


,Accuracy
0,97.22
1,97.78
2,70.73
3,78.79
4,66.10
5,93.10
6,97.67
7,65.96
8,67.50
9,78.57


Average Accuracy: 81.34200000000001




### Choose the smoothing parameter with the best average accuracy (k=5)

In [ ]:
LL_pred = pd.concat([pd.DataFrame(joint_log_likelihood[2]), df_testY], axis=1)
# LL_pred

In [ ]:
for index in range(10):
    i_most = LL_pred[LL_pred['label']==index].iloc[:,index].idxmax()
    i_least = LL_pred[LL_pred['label']==index].iloc[:,index].idxmin()
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,6))
    ax1.imshow(df_testX.iloc[i_most,:].values.reshape((32,32)), cmap='Greys')
    ax2.imshow(df_testX.iloc[i_least,:].values.reshape((32,32)), cmap='Greys')
    ax1.set_title('Visualizing most prototypical sample for {}'.format(index))
    ax2.set_title('Visualizing least prototypical sample for {}'.format(index))
    ax1.axis('off')
    ax2.axis('off')
    plt.subplots_adjust(wspace=0.5)
    plt.show()


In [ ]:
def plot_log_odds_ratio(c1, c2):
    f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,4))
    ax1 = sns.heatmap(feature_log_prob[2][c1].reshape((32,32)), ax=ax1, cmap='jet', xticklabels=False, yticklabels=False)
    ax2 = sns.heatmap(feature_log_prob[2][c2].reshape((32,32)), ax=ax2, cmap='jet', xticklabels=False, yticklabels=False)
    ax3 = sns.heatmap((feature_log_prob[2][c1] - feature_log_prob[2][c2]).reshape((32,32)), ax=ax3, cmap='jet', xticklabels=False, yticklabels=False)
    plt.subplots_adjust(wspace=0.3)
    plt.show()

In [ ]:
for i, cls in enumerate(zip([2,5,3,4], [8,9,9,7])):
    plot_log_odds_ratio(cls[0], cls[1])